In [60]:
from bs4 import BeautifulSoup
import requests
import json
import csv
import datetime

insight_news_info = []
cnt = 0 # 뉴스 number

# csv변환시키는 함수
def convert_to_csv(csv_name, fieldnames, infor):
    try:
        with open(f'./${csv_name}.csv', 'w', newline='') as c:
            fieldnames = fieldnames
            writer = csv.DictWriter(c, fieldnames=fieldnames) 
            writer.writeheader()

            for news in infor:
                print(news)
                writer.writerow(news) 
    except:
        pass

    
# Dict형태로 변환 함수
def dict_infor(**kwargs):
    news_info = []
    context = {}
    for k,v in kwargs.items():
        context[k] = v
    news_info.append(context)
#     print(news_info)
    return news_info



for num in range(1,2):
    response = requests.get(f'https://www.insight.co.kr/section/money?page={num}')
    soup = BeautifulSoup(response.text, 'html.parser')

    # 메인 페이지 기사 ul
    ul = soup.select_one('body > div.content > div > div.section-wrap > div.section-list > ul')
    # 메인 페이지 기사리스트들 li
    lis = ul.select('li')
    # 현재 카테고리 네임
    category = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > span')
    press = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > a').text
    # 기사 리스트 for문
    # 프리뷰 기사 img, title, date가져옴
    # 현재 카테고리 각각의 리스트에 함께 나오게 하기 위해 단순히 위에서 받은 new_category밑에다 써줌
    
    for li in lis:
        cnt += 1
        preview_img = li.select_one('li > div > a.section-list-article-image')
        preview_img = preview_img['style'].split("(")[1].split(')')[0]
        title = li.select_one('li > div > a.section-list-article-title').text.strip()
        news_category = category.text.split('>')[1].strip()
        date = li.select_one('li > div > span.section-list-article-byline')
        link = li.select_one('li > div > a.section-list-article-title')['href']
        code = link.split('/')[4]
        
        # code로 내용을 가지고 오는대신 다시한번 requests와 parser이용
        news_url = requests.get(link)
        news_url_html = BeautifulSoup(news_url.text, 'html.parser')
        
        # 세부 컨텐츠 안에 주요 내용 selector css         
        content = news_url_html.select_one('body > div.content > div > div.news-container > div.news-wrap > div.news-article > div.news-article-memo')
        # 세부 컨텐츠 안에 이미지, 이미지 여러개이므로 list에 담아줌
        news_image = content.select('img')
        image_url_list = []
        for image in news_image:
            image_url_list.append(image['src'])
        
        # 기자 이름이 날짜와 함께 나오는 경우가 있음 \".\" 이 있는 경우는 기자 네임이 있는 경우라 .을 기준으로 나누어서 날짜만 가져옴
        if '·' in date.text:
            date = date.text.strip().split('·')[1]

        else:
            date = date.text.strip()
        date_list = date.replace(' ','-').split('-')
        date_list = ' '.join(date_list).split()
        date_list = date_list[0]+'-'+date_list[1]+'-'+date_list[2]+' '+date_list[3]
        print(date_list)
        date_code = datetime.datetime.strptime(date_list,'%Y%m%d%H%M%S')
#         print(datetime.datetime.strptime(date_list,'%Y%m%d%H%M%S'))
        print(type(date_code))


        
        # 이하 똑같은 형식
        insight_news_info = dict_infor(num=cnt, press=press,news_code=code, news_category=news_category, date=date, preview_img=preview_img, title=title, content=content.text,img=image_url_list,ref=link, date_code= date_list)

# print(insight_news_info)
convert_to_csv('insight_news_infor',['num', 'press', 'news_code', 'news_category', 'date','preview_img','title', 'content', 'img', 'ref' ],insight_news_info)


2021-07-13 07:13:22


ValueError: time data '2021-07-13 07:13:22' does not match format '%Y%m%d%H%M%S'